In [2]:
import requests
from lxml import html
from bs4 import BeautifulSoup
import csv

USERNAME = "760b30b973"
PASSWORD = "pw29b7"
LOGIN_URL = "https://www.marklines.com/en/members/login"
URL = 'https://www.marklines.com/en/green_vehicles/case_top'

session_requests = requests.session()

# Get login csrf token
result = session_requests.get(LOGIN_URL)
tree = html.fromstring(result.text)
authenticity_token = list(set(tree.xpath("//input[@name='csrfToken']/@value")))[0]

# Create payload
payload = {
        "profiles.login.login_id": USERNAME, 
        "profiles.login.password": PASSWORD, 
        "csrfToken": authenticity_token
    }

# Perform login
result = session_requests.post(LOGIN_URL, data = payload, headers = dict(referer = LOGIN_URL))

session_requests = requests.session()

# Get login csrf token
result = session_requests.get(LOGIN_URL)
tree = html.fromstring(result.text)
authenticity_token = list(set(tree.xpath("//input[@name='csrfToken']/@value")))[0]

# Create payload
payload = {
        "profiles.login.login_id": USERNAME, 
        "profiles.login.password": PASSWORD, 
        "csrfToken": authenticity_token
    }

# Perform login
result = session_requests.post(LOGIN_URL, data = payload, headers = dict(referer = LOGIN_URL))

# Scrape url
result = session_requests.get(URL, headers = dict(referer = URL))
soup=BeautifulSoup(result.content,"html.parser")
contain = soup.find_all('tbody',attrs={'id':'table_body'})
tr = contain[0].find_all('tr')
#with open('green_vehicles1.csv','w') as outf :
    #create header name of csv file
    #outf.write('All_OEMs,Model_name,Status,Launch,Electric,Autonomous,Note\n')

def clean_string(var):
    var = str(var)
    var = var.rstrip()
    var = var.replace('\n','')
    return var

data = []    

for i in range(len(tr)):
    
    datum = {}
    td = tr[i].find_all('td')
    All_OEMs = td[1].text
    Model_name = td[2].text
    Status = td[3].text
    Launch = td[4].text
    Electric = td[5].text
    Autonomous = td[6].text
    Note = td[7].text
    #print('All OEMs:' + All_OEMs + ',Model name:' + Model_name + ',Status:' + Status + ',Launch:' + Launch + ',Electric:' + Electric + ',Autonomous:' + Autonomous + ',Note:' + Note)
    #outf.write(All_OEMs + ',' + Model_name + ',' + Status + ',' + Launch + ',' + Electric + ',' + Autonomous + ',' + Note + '\n')
    datum['All OEMs'] = All_OEMs
    datum['Model_name'] = clean_string(Model_name) 
    datum['Status'] = Status
    datum['Launch'] = Launch
    datum['Electric'] = Electric
    datum['Autonomous'] = Autonomous
    datum['Note'] = Note
    data.append(datum)
    

In [3]:
data[0]

{'All OEMs': 'USA GM',
 'Model_name': 'Chevrolet Monza Redline/RS',
 'Status': 'Launched',
 'Launch': '2020',
 'Electric': 'HV',
 'Autonomous': 'Level 1',
 'Note': 'A compact sedan (48V)'}

In [4]:
from pymongo import MongoClient

In [5]:
client = MongoClient()

In [6]:
client = MongoClient('localhost',27017)

In [7]:
database = client.intellcap

In [8]:
for row in data:
    database.vehicle.insert_one(row)